In [36]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.models import resnet18
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import pytorch_lightning as pl
import pandas as pd
import numpy as np
import torch.nn.functional as F

In [2]:
class NetVLAD(nn.Module):
    """NetVLAD layer implementation"""

    def __init__(self, num_clusters=64, dim=128, alpha=100.0,
                 normalize_input=True):
        """
        Args:
            num_clusters : int
                The number of clusters
            dim : int
                Dimension of descriptors
            alpha : float
                Parameter of initialization. Larger value is harder assignment.
            normalize_input : bool
                If true, descriptor-wise L2 normalization is applied to input.
        """
        super(NetVLAD, self).__init__()
        self.num_clusters = num_clusters
        self.dim = dim
        self.alpha = alpha
        self.normalize_input = normalize_input
        self.conv = nn.Conv2d(dim, num_clusters, kernel_size=(1, 1), bias=True)
        self.centroids = nn.Parameter(torch.rand(num_clusters, dim))
        self._init_params()

    def _init_params(self):
        self.conv.weight = nn.Parameter(
            (2.0 * self.alpha * self.centroids).unsqueeze(-1).unsqueeze(-1)
        )
        self.conv.bias = nn.Parameter(
            - self.alpha * self.centroids.norm(dim=1)
        )

    def forward(self, x):
        N, C = x.shape[:2]

        if self.normalize_input:
            x = F.normalize(x, p=2, dim=1)  # across descriptor dim

        # soft-assignment
        soft_assign = self.conv(x).view(N, self.num_clusters, -1)
        soft_assign = F.softmax(soft_assign, dim=1)

        x_flatten = x.view(N, C, -1)
        
        # calculate residuals to each clusters
        residual = x_flatten.expand(self.num_clusters, -1, -1, -1).permute(1, 0, 2, 3) - \
            self.centroids.expand(x_flatten.size(-1), -1, -1).permute(1, 2, 0).unsqueeze(0)
        residual *= soft_assign.unsqueeze(2)
        vlad = residual.sum(dim=-1)

        vlad = F.normalize(vlad, p=2, dim=2)  # intra-normalization
        vlad = vlad.view(x.size(0), -1)  # flatten
        vlad = F.normalize(vlad, p=2, dim=1)  # L2 normalize

        return vlad

        
class EmbedNet(nn.Module):
    def __init__(self, base_model, net_vlad):
        super(EmbedNet, self).__init__()
        self.base_model = base_model
        self.net_vlad = net_vlad

    def forward(self, x):
        x = self.base_model(x)
        embedded_x = self.net_vlad(x)
        return embedded_x

In [3]:
transform = A.Compose([
    A.CenterCrop(800, 800),
    A.Resize(250, 250),
    A.RandomCrop(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])
criterion = nn.TripletMarginLoss()

In [4]:
class CPDataset(Dataset):
    def __init__(self, transform, data_dir, train=True):
        self.data_dir = data_dir
        self.train = train
        if self.train:
            self.df = pd.read_csv('{}/train.csv'.format(data_dir))
            self.prefix = '../CGU-Pipeline-Dataset/train/'
        else:
            self.df = pd.read_csv('{}/test.csv'.format(data_dir))
            self.prefix = '../CGU-Pipeline-Dataset/test/'
        self.index = self.df.index.values
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):\
        # anchor_img
        anchor_img = np.array(Image.open(self.prefix + self.df.Filename[idx]))
        # anchor_label
        anchor_label = self.df.Class[idx]    
        if self.train:
            # positive_img
            positive_list = self.index[self.index!=idx][self.df.Class[self.index!=idx]==anchor_label]
            positive_item = np.random.choice(positive_list)
            positive_img = np.array(Image.open(self.prefix + self.df.Filename[positive_item]))
            # negative_img
            negative_list = self.index[self.index!=idx][self.df.Class[self.index!=idx]!=anchor_label]
            negative_item = np.random.choice(negative_list)
            negative_img = np.array(Image.open(self.prefix + self.df.Filename[negative_item]))
            # transform
            anchor_img = self.transform(image=anchor_img)['image']
            positive_img = self.transform(image=positive_img)['image']
            negative_img = self.transform(image=negative_img)['image']
            return anchor_img, positive_img, negative_img, anchor_label
        else:
            anchor_img = self.transform(image=anchor_img)['image']
            return anchor_img, anchor_label


In [46]:
class CPDatamodule(pl.LightningDataModule):
    def __init__(self, transform, data_dir, batch_size=32):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transform
    def setup(self, stage = None):
        self.cpd_train = CPDataset(self.transform, self.data_dir, train=True)
        train_size = int(len(self.cpd_train)*0.7)
        test_size = len(self.cpd_train) - train_size 
        self.cpd_train, self.cpd_val = random_split(self.cpd_train, [train_size, test_size])
        self.cpd_test = CPDataset(self.transform, self.data_dir, train=False)
    def train_dataloader(self):
        return DataLoader(self.cpd_train, batch_size=self.batch_size)
    def val_dataloader(self):
        return DataLoader(self.cpd_val, batch_size=self.batch_size)
    def test_dataloader(self):
        return DataLoader(self.cpd_test, batch_size=self.batch_size)

In [75]:
class LitNetVlad(pl.LightningModule):
    def __init__(self):
        super().__init__()
        encoder = resnet18(pretrained=True)
        base_model = nn.Sequential(
            encoder.conv1,
            encoder.bn1,
            encoder.relu,
            encoder.maxpool,
            encoder.layer1,
            encoder.layer2,
            encoder.layer3,
            encoder.layer4,
        )
        dim = list(base_model.parameters())[-1].shape[0]  # last channels (512)
        net_vlad = NetVLAD(num_clusters=32, dim=dim, alpha=1.0)
        self.model = EmbedNet(base_model, net_vlad)
        self.criterion = nn.TripletMarginLoss()
    def forward(self, x):
        return self.model(x)
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01)
        return optimizer
    def training_step(self, train_batch, batch_idx):
        anc, pos, neg, label = train_batch
        anc_out, pos_out, neg_out = self.model(anc), self.model(pos), self.model(neg)
        loss = self.criterion(anc_out, pos_out, neg_out)
        self.logger.experiment.add_scalar('train_batch_loss', loss, self.global_step)
        return loss
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        self.logger.experiment.add_scalar('train_epoch_loss', avg_loss, self.current_epoch)
                            
    def validation_step(self, val_batch, batch_idx):
        anc, pos, neg, label = val_batch
        anc_out, pos_out, neg_out = self.model(anc), self.model(pos), self.model(neg)
        loss = self.criterion(anc_out, pos_out, neg_out)
        self.logger.experiment.add_scalar('val_batch_loss', loss, self.global_step)
        return loss
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x for x in outputs]).mean()
        self.logger.experiment.add_scalar('val_epoch_loss', avg_loss, self.current_epoch)

In [ ]:
resume = True
cpd = CPDatamodule(transform=transform, data_dir='../CGU-Pipeline-Dataset')
model = LitNetVlad()
logger = pl.loggers.TensorBoardLogger('lightning-logs', name = 'netvlad')
if resume:
    trainer = pl.Trainer(gpus=1, precision=16, limit_train_batches=0.5, logger=logger, resume_from_checkpoint='lightning-logs/netvlad/version_3/checkpoints/epoch=143-step=1871.ckpt')
else:
    trainer = pl.Trainer(gpus=1, precision=16, limit_train_batches=0.5, logger=logger)
trainer.fit(model, cpd)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | model     | EmbedNet          | 11.2 M
1 | criterion | TripletMarginLoss | 0     
------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.837    Total estimated model params size (MB)
Restored states from the checkpoint file at lightning-logs/netvlad/version_3/checkpoints/epoch=143-step=1871.ckpt


Validation sanity check: 0it [00:00, ?it/s]

C:\Users\vlsi71\anaconda3\envs\torch\lib\site-packages\pytorch_lightning\trainer\data_loading.py:102: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\vlsi71\anaconda3\envs\torch\lib\site-packages\pytorch_lightning\trainer\data_loading.py:102: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]